In [27]:
import torch
torch.cuda.empty_cache()


# ENTRENAMIENTO

In [1]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)

# ================================
# 1. Carga y preparación de datos
# ================================
# Cargar dataset de entrenamiento con etiquetas
df = pd.read_csv("/home/adrian/Escritorio/DeepSexist/DatasetManagement/EXIST2025DatasetV0.3/EXIST 2025 Videos Dataset/training/EXIST2025_training_task3_1_DISCARD.csv")
df = df.rename(columns={"label_task_3_1_merged": "label"})
df = df.drop(columns=["Unnamed: 0"])
df["label"] = df["label"].astype(int)

from sklearn.model_selection import train_test_split
train_df, eval_df = train_test_split(df, test_size=0.1, stratify=df["label"], random_state=42)

train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)
# ================================
# 2. Tokenización
# ================================
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ================================
# 3. Carga del modelo
# ================================
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# ================================
# 4. Argumentos de entrenamiento
# ================================
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    num_train_epochs=5,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",  # evita errores si no usas wandb
)

# ================================
# 5. Entrenamiento
# ================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

# ================================
# 6. Predicción sobre el test
# ================================
# Cargar y limpiar test sin etiquetas
df_test = pd.read_json(
    "/home/adrian/Escritorio/DeepSexist/DatasetManagement/EXIST2025DatasetV0.3/EXIST 2025 Videos Dataset/test/EXIST2025_test_clean.json",
    orient="index"
)
df_test.reset_index(drop=True, inplace=True)
df_test = df_test.drop(columns=[
    "id_Tiktok", "video", "url", "annotators", "gender_annotators",
    "split", "lang", "path_video", "number_annotators"
])

test_dataset = Dataset.from_pandas(df_test)
test_dataset = test_dataset.map(preprocess_function, batched=True)

# Predicción
predictions = trainer.predict(test_dataset)
predicted_labels = predictions.predictions.argmax(-1)

# Guardar resultados
df_test["predicted_label"] = predicted_labels
#df_test.to_csv("predicciones_test.csv", index=False)
print("✅ Predicciones guardadas en 'predicciones_test.csv'")


Map:   0%|          | 0/2257 [00:00<?, ? examples/s]

Map:   0%|          | 0/251 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_839505/1161705695.py:59: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


  0%|          | 0/1415 [00:00<?, ?it/s]

{'loss': 0.7301, 'grad_norm': 12.481428146362305, 'learning_rate': 4.9646643109540636e-05, 'epoch': 0.04}
{'loss': 0.7122, 'grad_norm': 4.158790588378906, 'learning_rate': 4.929328621908128e-05, 'epoch': 0.07}
{'loss': 0.7232, 'grad_norm': 2.6054975986480713, 'learning_rate': 4.893992932862191e-05, 'epoch': 0.11}
{'loss': 0.7046, 'grad_norm': 4.183778285980225, 'learning_rate': 4.858657243816255e-05, 'epoch': 0.14}
{'loss': 0.7508, 'grad_norm': 4.903720855712891, 'learning_rate': 4.823321554770318e-05, 'epoch': 0.18}
{'loss': 0.7072, 'grad_norm': 5.647780418395996, 'learning_rate': 4.787985865724382e-05, 'epoch': 0.21}
{'loss': 0.7547, 'grad_norm': 3.0035829544067383, 'learning_rate': 4.752650176678445e-05, 'epoch': 0.25}
{'loss': 0.6906, 'grad_norm': 4.578444480895996, 'learning_rate': 4.717314487632509e-05, 'epoch': 0.28}
{'loss': 0.7093, 'grad_norm': 3.5777251720428467, 'learning_rate': 4.6819787985865726e-05, 'epoch': 0.32}
{'loss': 0.6839, 'grad_norm': 2.6623239517211914, 'learnin

Map:   0%|          | 0/674 [00:00<?, ? examples/s]

  0%|          | 0/85 [00:00<?, ?it/s]

✅ Predicciones guardadas en 'predicciones_test.csv'


In [5]:
df_test.to_csv('/home/adrian/Escritorio/DeepSexist/TrainingBooks/results/Bicheando/prediccion.csv')

In [8]:
df_test

,id_EXIST,text,predicted_label
0,320001,si me siguió te crece +3cm me echan del instit...,0
1,320002,humor gente un cardumen de mujeres en su hábit...,1
2,320003,"auronplay diciendo ""un generador de niños, que...",1
3,320004,no se vé face_with_peeking_eye mi amigo reacc...,1
4,320005,"¿y las promesas? siempre mienten, fin. el tie...",0
...,...,...,...
669,320300,no somos así estamos para que hagan un filtro ...,0
670,320301,"no eres salvavidas de nadie, sólo de ti misma ...",1
671,320302,amate como el temach y ama como farid dieck ...,1
672,320303,dos filosofías muy diferentes pero q hablan de...,1


In [7]:
df_test_json = df_test.to_json()

df_test_json

'{"id_EXIST":{"0":320001,"1":320002,"2":320003,"3":320004,"4":320005,"5":320006,"6":320007,"7":320008,"8":320009,"9":320010,"10":320011,"11":320012,"12":320013,"13":320014,"14":320015,"15":320016,"16":320017,"17":320018,"18":320019,"19":320020,"20":320021,"21":320022,"22":320023,"23":320024,"24":320025,"25":320026,"26":320027,"27":320028,"28":320029,"29":320030,"30":320031,"31":320032,"32":320033,"33":320034,"34":320035,"35":320036,"36":320037,"37":320038,"38":320039,"39":320040,"40":320041,"41":320042,"42":320043,"43":320044,"44":320045,"45":320046,"46":320047,"47":320048,"48":320049,"49":320050,"50":320051,"51":320052,"52":320053,"53":320054,"54":320055,"55":320056,"56":320057,"57":320058,"58":320059,"59":320060,"60":320061,"61":320062,"62":320063,"63":320064,"64":320065,"65":320066,"66":320067,"67":320068,"68":320069,"69":320070,"70":320071,"71":320072,"72":320073,"73":320074,"74":320075,"75":320076,"76":320077,"77":320078,"78":320079,"79":320080,"80":320081,"81":320082,"82":320083,

In [ ]:
import pandas as pd
import json

# Cargar el CSV
#df = pd.read_csv("predicciones_test.csv")

# Convertir los valores
output = []
for _, row in df_test.iterrows():
    entry = {
        "test_case": "EXIST2025",
        "id": str(row["id_EXIST"]),
        "value": "YES" if row["predicted_label"] == 1 else "NO"
    }
    output.append(entry)

# Guardar como JSON
with open("/home/adrian/Escritorio/DeepSexist/TrainingBooks/results/Bicheando/predicciones_formato_EXIST2025.json", "w") as f:
    json.dump(output, f, indent=2)

print("✅ Archivo guardado como 'predicciones_formato_EXIST2025.json'")


NameError: name 'df_test' is not defined

In [10]:
df_json = pd.read_json('/home/adrian/Escritorio/DeepSexist/TrainingBooks/results/Bicheando/predicciones_formato_EXIST2025.json')

df_json

,test_case,id,value
0,EXIST2025,320001,NO
1,EXIST2025,320002,YES
2,EXIST2025,320003,YES
3,EXIST2025,320004,YES
4,EXIST2025,320005,NO
...,...,...,...
669,EXIST2025,320300,NO
670,EXIST2025,320301,YES
671,EXIST2025,320302,YES
672,EXIST2025,320303,YES


Formato de la subida usando *PyEvAll*

In [ ]:
from pyevall.evaluation import PyEvALLEvaluation
from pyevall.utils.utils import PyEvALLUtils

predictions = "//home/adrian/Escritorio/DeepSexist/TrainingBooks/results/exist2025_I2C-UHU-Sirius/task3_1_hard_I2C-UHU-Sirius_1/bert-base-multilingual-cased.json"
#gold = "/home/adrian/Escritorio/DeepSexist/TrainingBooks/results/exist2025_I2C-UHU-Sirius/task3_1_hard_I2C-UHU-Sirius_1/prediction.json"
gold = "/home/adrian/Escritorio/DeepSexist/DatasetManagement/EXIST2025DatasetV0.3/evaluation/golds/EXIST2025_training_task3_1_gold_hard.json"
test = PyEvALLEvaluation()
params= dict()
params[PyEvALLUtils.PARAM_REPORT]= PyEvALLUtils.PARAM_OPTION_REPORT_EMBEDDED
metrics=["ICM", "ICMNorm" ,"FMeasure"]
report= test.evaluate(predictions, gold, metrics, **params)
report.print_report()

2025-05-17 22:01:36,542 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM', 'ICMNorm', 'FMeasure']
2025-05-17 22:01:36,618 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-17 22:01:36,858 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM Normalized evaluation method
2025-05-17 22:01:36,859 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-17 22:01:37,101 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-17 22:01:37,510 - pyevall.metrics.metrics - INFO -             evaluate() - Executing fmeasure evaluation method
{
  "metrics": {
    "ICM": {
      "name": "Information Contrast model",
      "acronym": "ICM",
      "description": "Coming soon!",
      "status": "OK",
      "results": {
        "test_cases": [{
          "name": "EXIST2025",
          "average": -0.99875

: 